In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support

## Own code 
import import_ipynb
import metadata_options
import models_nn

Using TensorFlow backend.


importing Jupyter notebook from metadata_options.ipynb
importing Jupyter notebook from models_nn.ipynb


In [2]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')
df = df.dropna()

In [3]:
# Edit option choices here
throughput_option = 'TP1'
worktime_option = 'WT1'
pc_agreement_option = 'PC3'
textlength_option = 'TL2'
special_option = 'SP1'
k_option_for_tp = 3

df_throughput, df_worktime, df_agreement, df_textlength, df_special = metadata_options.set_OHE_pipeline_options(df, throughput_option, worktime_option, pc_agreement_option, textlength_option, special_option, k_option_for_tp)

TP1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
WT1: weighted by 1 average per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
PC3: weighted by 1 PC agreement weight per annotation in each OHE, i.e. (a, b, c, d) -> (w1*a, w2*b, w3*c, w4*d)
TL2: weighted by 1 normalised number of words per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)
SP1: weighted by average of TP1 and TP2 per set of OHE, i.e. (a, b, c, d) -> (w*a, w*b, w*c, w*d)


In [4]:
######################################
## Model Options ##
######################################
# options: lstm, cnn, lstm-attn

model_name = 'lstm-attn'
models_nn.MODEL_NAME = model_name

In [5]:
df.head()

,Unnamed: 0,Input.sentence_id,HITId,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,Input.speaker,Input.reply_to,...,rcmod,rel,root,tmod,xcomp,xsubj,Answer.1gamemove.yes_pc_agree,Answer.2reasoning.yes_pc_agree,Answer.4shareinformation.yes_pc_agree,Answer.3rapport.yes_pc_agree
0,5,11,3MG8450X2OASXZ0WO9O5AH70GU3UPA,Game1-italy-germany,Train,Game1-italy-germany-3,87,It seems like there are a lot of ways that cou...,germany-Game1,Game1-italy-germany-2,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.8,1.0,0.8
1,6,12,38G0E1M85M552JXSALX4G9WI2I6UVX,Game1-italy-germany,Train,Game1-italy-germany-7,117,"Sorry Italy I've been away doing, um, German t...",germany-Game1,Game1-italy-germany-6,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.6,0.8,0.8
2,7,14,3HYV4299H0WQ2B4TCS7PKDQ75WHE81,Game1-italy-germany,Train,Game1-italy-germany-8,119,"I don't think I'm ready to go for that idea, h...",germany-Game1,Game1-italy-germany-7,...,0.0,0.0,1.0,0.0,1.0,0.0,0.8,0.6,0.6,0.6
3,8,15,3XU9MCX6VOC4P079IHIO9TCNYLGR2P,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am pretty conflicted about whether to guess ...,italy-Game1,Game1-italy-germany-8,...,0.0,0.0,1.0,0.0,0.0,0.0,0.8,0.8,0.8,0.8
4,9,16,3FVBZG9CLJEK4WQS7P2GC1H2EEQH0Q,Game1-italy-germany,Train,Game1-italy-germany-9,121,I am going to take it literally and say even ...,italy-Game1,Game1-italy-germany-8,...,0.0,0.0,1.0,0.0,2.0,1.0,0.8,0.8,1.0,1.0


In [6]:
# New train_test_split using Stratified Shaffled Splits
y = df["Input.deception_quadrant"].copy()
X = df.drop(["Input.deception_quadrant"], axis=1)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
splits_generator = sss.split(X, y)

for train_idx, test_idx in splits_generator:
    indices_train = train_idx
    indices_test = test_idx

train = df.take(indices_train)
test = df.take(indices_test)

In [7]:
print(df.shape, train.shape, test.shape)

(11366, 863) (9092, 863) (2274, 863)


In [8]:
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.23.2


In [9]:
new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].to_numpy()
deception_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_deception),
                                                 y_train_deception)
deception_class_weight_dict = dict(enumerate(deception_class_weights))

y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
rapport_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_rapport),
                                                 y_train_rapport)
rapport_class_weight_dict = dict(enumerate(rapport_class_weights))

y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
share_info_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_share_information),
                                                 y_train_share_information)
share_info_class_weight_dict = dict(enumerate(share_info_class_weights))

y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
reasoning_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_reasoning),
                                                 y_train_reasoning)
reasoning_class_weight_dict = dict(enumerate(reasoning_class_weights))

y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()
gamemove_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_gamemove),
                                                 y_train_gamemove)
gamemove_class_weight_dict = dict(enumerate(gamemove_class_weights))

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[1 1 1 ... 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [10]:
print("Class weights")
print("Deception: {} \nRapport: {} \nShare Information: {} \nReasoning: {} \nGamemove: {}".format(deception_class_weight_dict,
                                                                                                  rapport_class_weight_dict,
                                                                                                  share_info_class_weight_dict,
                                                                                                  reasoning_class_weight_dict,
                                                                                                  gamemove_class_weight_dict))

Class weights
Deception: {0: 10.192825112107624, 1: 0.525792273883877} 
Rapport: {0: 3.582348305752561, 1: 0.5811069922024799} 
Share Information: {0: 3.117969821673525, 1: 0.59549384333246} 
Reasoning: {0: 2.9596354166666665, 1: 0.6016410799364743} 
Gamemove: {0: 7.048062015503876, 1: 0.5381792352314431}


In [11]:
X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [12]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [13]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

### Construct individual  models

In [14]:
# Rapport model
rapport_model = models_nn.create_nn_model()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                              models_nn.recall_m, models_nn.precision_m])
rapport_model.fit(X_train,y_train_rapport,
                  batch_size=128,
                  epochs=50,
                  validation_data=(X_test, y_test_rapport), 
                  class_weight=rapport_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 47ms/step - loss: 0.6933 - accuracy: 0.7740 - f1_m: 0.8211 - recall_m: 0.8852 - precision_m: 0.7740 - val_loss: 0.6918 - val_accuracy: 0.8712 - val_f1_m: 0.9305 - val_recall_m: 1.0000 - val_precision_m: 0.8704
Epoch 2/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6932 - accuracy: 0.6671 - f1_m: 0.6960 - recall_m: 0.7416 - precision_m: 0.7099 - val_loss: 0.6918 - val_accuracy: 0.8712 - val_f1_m: 0.9305 - val_recall_m: 1.0000 - val_precision_m: 0.8704
Epoch 3/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6926 - accuracy: 0.6948 - f1_m: 0.7850 - recall_m: 0.7616 - precision_m: 0.8760 - val_loss: 0.6637 - val_accuracy: 0.8712 - val_f1_m: 0.9305 - val_recall_m: 1.0000 - val_precision_m: 0.8704
Epoch 4/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6904 - accuracy: 0.7951 - f1_m: 0.8823 - recall_m: 0.9037 - precision_m: 0.8680 - val_loss: 0.6545 - val_accuracy: 0.6135 - val_f1

Epoch 32/50
72/72 [==============================] - 3s 38ms/step - loss: 0.4378 - accuracy: 0.7498 - f1_m: 0.8295 - recall_m: 0.7261 - precision_m: 0.9745 - val_loss: 0.8173 - val_accuracy: 0.5391 - val_f1_m: 0.6758 - val_recall_m: 0.5566 - val_precision_m: 0.8638
Epoch 33/50
72/72 [==============================] - 3s 38ms/step - loss: 0.4271 - accuracy: 0.7571 - f1_m: 0.8391 - recall_m: 0.7411 - precision_m: 0.9749 - val_loss: 0.6712 - val_accuracy: 0.6526 - val_f1_m: 0.7773 - val_recall_m: 0.7037 - val_precision_m: 0.8700
Epoch 34/50
72/72 [==============================] - 3s 38ms/step - loss: 0.4259 - accuracy: 0.7643 - f1_m: 0.8457 - recall_m: 0.7513 - precision_m: 0.9736 - val_loss: 0.7408 - val_accuracy: 0.6099 - val_f1_m: 0.7412 - val_recall_m: 0.6476 - val_precision_m: 0.8687
Epoch 35/50
72/72 [==============================] - 3s 39ms/step - loss: 0.4154 - accuracy: 0.7736 - f1_m: 0.8525 - recall_m: 0.7597 - precision_m: 0.9757 - val_loss: 1.0101 - val_accuracy: 0.4639 - va

In [15]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [16]:
# Game move model
gamemove_model = models_nn.create_nn_model()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, 
                                                                               models_nn.recall_m, models_nn.precision_m])
gamemove_model.fit(X_train,
                   y_train_gamemove,
                   batch_size=128,
                   epochs=50,
                   validation_data=(X_test, y_test_gamemove), 
                   class_weight=gamemove_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 46ms/step - loss: 0.6933 - accuracy: 0.2931 - f1_m: 0.2508 - recall_m: 0.2573 - precision_m: 0.2570 - val_loss: 0.6922 - val_accuracy: 0.9376 - val_f1_m: 0.9675 - val_recall_m: 1.0000 - val_precision_m: 0.9372
Epoch 2/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6930 - accuracy: 0.6199 - f1_m: 0.6823 - recall_m: 0.6464 - precision_m: 0.8431 - val_loss: 0.6827 - val_accuracy: 0.5365 - val_f1_m: 0.6817 - val_recall_m: 0.5329 - val_precision_m: 0.9503
Epoch 3/50
72/72 [==============================] - 3s 39ms/step - loss: 0.6939 - accuracy: 0.7883 - f1_m: 0.8336 - recall_m: 0.8197 - precision_m: 0.9104 - val_loss: 0.6651 - val_accuracy: 0.6262 - val_f1_m: 0.7599 - val_recall_m: 0.6337 - val_precision_m: 0.9509
Epoch 4/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6823 - accuracy: 0.4526 - f1_m: 0.5807 - recall_m: 0.4342 - precision_m: 0.9545 - val_loss: 0.6483 - val_accuracy: 0.4952 - val_f1

Epoch 32/50
72/72 [==============================] - 3s 38ms/step - loss: 0.4459 - accuracy: 0.7542 - f1_m: 0.8467 - recall_m: 0.7496 - precision_m: 0.9858 - val_loss: 0.4555 - val_accuracy: 0.7907 - val_f1_m: 0.8812 - val_recall_m: 0.8312 - val_precision_m: 0.9396
Epoch 33/50
72/72 [==============================] - 3s 38ms/step - loss: 0.4541 - accuracy: 0.7644 - f1_m: 0.8500 - recall_m: 0.7558 - precision_m: 0.9844 - val_loss: 0.4160 - val_accuracy: 0.8193 - val_f1_m: 0.8997 - val_recall_m: 0.8653 - val_precision_m: 0.9379
Epoch 34/50
72/72 [==============================] - 3s 39ms/step - loss: 0.4362 - accuracy: 0.7678 - f1_m: 0.8575 - recall_m: 0.7644 - precision_m: 0.9861 - val_loss: 0.5798 - val_accuracy: 0.6939 - val_f1_m: 0.8149 - val_recall_m: 0.7210 - val_precision_m: 0.9391
Epoch 35/50
72/72 [==============================] - 3s 38ms/step - loss: 0.4300 - accuracy: 0.7621 - f1_m: 0.8481 - recall_m: 0.7503 - precision_m: 0.9878 - val_loss: 0.4201 - val_accuracy: 0.8026 - va

In [17]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [18]:
# Reasoning model
reasoning_model = models_nn.create_nn_model()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
reasoning_model.fit(X_train,
                    y_train_reasoning,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_reasoning), 
                    class_weight=reasoning_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 46ms/step - loss: 0.6932 - accuracy: 0.7644 - f1_m: 0.8090 - recall_m: 0.8880 - precision_m: 0.7480 - val_loss: 0.6926 - val_accuracy: 0.8219 - val_f1_m: 0.9011 - val_recall_m: 1.0000 - val_precision_m: 0.8212
Epoch 2/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6932 - accuracy: 0.7496 - f1_m: 0.8182 - recall_m: 0.8821 - precision_m: 0.8079 - val_loss: 0.6938 - val_accuracy: 0.1781 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 3/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6946 - accuracy: 0.7123 - f1_m: 0.7743 - recall_m: 0.8224 - precision_m: 0.7577 - val_loss: 0.6919 - val_accuracy: 0.5475 - val_f1_m: 0.6645 - val_recall_m: 0.5489 - val_precision_m: 0.8460
Epoch 4/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6922 - accuracy: 0.6362 - f1_m: 0.7509 - recall_m: 0.6857 - precision_m: 0.8530 - val_loss: 0.6824 - val_accuracy: 0.5

Epoch 32/50
72/72 [==============================] - 3s 39ms/step - loss: 0.4459 - accuracy: 0.7586 - f1_m: 0.8221 - recall_m: 0.7293 - precision_m: 0.9491 - val_loss: 0.7997 - val_accuracy: 0.6258 - val_f1_m: 0.7490 - val_recall_m: 0.6861 - val_precision_m: 0.8268
Epoch 33/50
72/72 [==============================] - 3s 41ms/step - loss: 0.4341 - accuracy: 0.7692 - f1_m: 0.8445 - recall_m: 0.7561 - precision_m: 0.9633 - val_loss: 0.9348 - val_accuracy: 0.5255 - val_f1_m: 0.6448 - val_recall_m: 0.5286 - val_precision_m: 0.8304
Epoch 34/50
72/72 [==============================] - 3s 46ms/step - loss: 0.4324 - accuracy: 0.7736 - f1_m: 0.8463 - recall_m: 0.7570 - precision_m: 0.9653 - val_loss: 0.7275 - val_accuracy: 0.6464 - val_f1_m: 0.7644 - val_recall_m: 0.7034 - val_precision_m: 0.8394
Epoch 35/50
72/72 [==============================] - 3s 39ms/step - loss: 0.4173 - accuracy: 0.7854 - f1_m: 0.8564 - recall_m: 0.7710 - precision_m: 0.9686 - val_loss: 0.8767 - val_accuracy: 0.5814 - va

In [19]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [20]:
# Share Information model
shareinfo_model = models_nn.create_nn_model()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
shareinfo_model.fit(X_train,
                    y_train_share_information,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, y_test_share_information), 
                    class_weight=share_info_class_weight_dict)

Epoch 1/50
72/72 [==============================] - 3s 47ms/step - loss: 0.6932 - accuracy: 0.3583 - f1_m: 0.2826 - recall_m: 0.3038 - precision_m: 0.2697 - val_loss: 0.6904 - val_accuracy: 0.8443 - val_f1_m: 0.9147 - val_recall_m: 1.0000 - val_precision_m: 0.8436
Epoch 2/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6932 - accuracy: 0.8285 - f1_m: 0.9015 - recall_m: 0.9826 - precision_m: 0.8425 - val_loss: 0.6913 - val_accuracy: 0.8443 - val_f1_m: 0.9147 - val_recall_m: 1.0000 - val_precision_m: 0.8436
Epoch 3/50
72/72 [==============================] - 3s 38ms/step - loss: 0.6932 - accuracy: 0.7713 - f1_m: 0.8406 - recall_m: 0.9022 - precision_m: 0.8265 - val_loss: 0.7123 - val_accuracy: 0.1557 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 4/50
72/72 [==============================] - 3s 39ms/step - loss: 0.6910 - accuracy: 0.6175 - f1_m: 0.6951 - recall_m: 0.6484 - precision_m: 0.7870 - val_loss: 0.6942 - val_accuracy: 0.4

Epoch 32/50
72/72 [==============================] - 3s 39ms/step - loss: 0.4369 - accuracy: 0.7201 - f1_m: 0.7986 - recall_m: 0.6806 - precision_m: 0.9743 - val_loss: 0.7627 - val_accuracy: 0.6588 - val_f1_m: 0.7803 - val_recall_m: 0.7244 - val_precision_m: 0.8482
Epoch 33/50
72/72 [==============================] - 3s 39ms/step - loss: 0.4308 - accuracy: 0.7264 - f1_m: 0.8088 - recall_m: 0.6960 - precision_m: 0.9718 - val_loss: 0.7978 - val_accuracy: 0.5185 - val_f1_m: 0.6488 - val_recall_m: 0.5295 - val_precision_m: 0.8415
Epoch 34/50
72/72 [==============================] - 3s 39ms/step - loss: 0.4306 - accuracy: 0.7258 - f1_m: 0.8072 - recall_m: 0.6920 - precision_m: 0.9760 - val_loss: 0.7723 - val_accuracy: 0.5884 - val_f1_m: 0.7172 - val_recall_m: 0.6231 - val_precision_m: 0.8475
Epoch 35/50
72/72 [==============================] - 3s 38ms/step - loss: 0.4157 - accuracy: 0.7405 - f1_m: 0.8144 - recall_m: 0.7038 - precision_m: 0.9754 - val_loss: 0.8617 - val_accuracy: 0.5928 - va

In [21]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [22]:
# Deception model
deception_model = models_nn.create_nn_model()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', models_nn.f1_m, models_nn.recall_m, models_nn.precision_m])
deception_model.fit(X_train,
                    y_train_deception,
                    epochs=50,
                    validation_data=(X_test, y_test_deception), 
                    class_weight=deception_class_weight_dict)

Epoch 1/50
285/285 [==============================] - 8s 28ms/step - loss: 0.6935 - accuracy: 0.5894 - f1_m: 0.5922 - recall_m: 0.6054 - precision_m: 0.5853 - val_loss: 0.6911 - val_accuracy: 0.9507 - val_f1_m: 0.9747 - val_recall_m: 1.0000 - val_precision_m: 0.9514
Epoch 2/50
285/285 [==============================] - 8s 27ms/step - loss: 0.6938 - accuracy: 0.8022 - f1_m: 0.8145 - recall_m: 0.8308 - precision_m: 0.8086 - val_loss: 0.7022 - val_accuracy: 0.0493 - val_f1_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00
Epoch 3/50
285/285 [==============================] - 8s 27ms/step - loss: 0.6944 - accuracy: 0.4649 - f1_m: 0.4702 - recall_m: 0.4610 - precision_m: 0.5482 - val_loss: 0.5755 - val_accuracy: 0.9507 - val_f1_m: 0.9747 - val_recall_m: 1.0000 - val_precision_m: 0.9514
Epoch 4/50
285/285 [==============================] - 8s 27ms/step - loss: 0.6847 - accuracy: 0.7060 - f1_m: 0.7751 - recall_m: 0.7239 - precision_m: 0.9428 - val_loss: 0.5252 - val_accur

285/285 [==============================] - 8s 27ms/step - loss: 0.5882 - accuracy: 0.7227 - f1_m: 0.8023 - recall_m: 0.7227 - precision_m: 0.9801 - val_loss: 1.2317 - val_accuracy: 0.2515 - val_f1_m: 0.3585 - val_recall_m: 0.2244 - val_precision_m: 0.9605
Epoch 32/50
285/285 [==============================] - 8s 27ms/step - loss: 0.5768 - accuracy: 0.6992 - f1_m: 0.8010 - recall_m: 0.7002 - precision_m: 0.9782 - val_loss: 0.6168 - val_accuracy: 0.6750 - val_f1_m: 0.7981 - val_recall_m: 0.6913 - val_precision_m: 0.9516
Epoch 33/50
285/285 [==============================] - 8s 27ms/step - loss: 0.4910 - accuracy: 0.7587 - f1_m: 0.8449 - recall_m: 0.7589 - precision_m: 0.9850 - val_loss: 0.3538 - val_accuracy: 0.9195 - val_f1_m: 0.9579 - val_recall_m: 0.9649 - val_precision_m: 0.9523
Epoch 34/50
285/285 [==============================] - 8s 27ms/step - loss: 0.5315 - accuracy: 0.7437 - f1_m: 0.8344 - recall_m: 0.7443 - precision_m: 0.9815 - val_loss: 0.6011 - val_accuracy: 0.6741 - val_f1

In [23]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)

### One hot encodings

In [24]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [25]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

## Weighted against Throughput, WorkTime, PC Agreement & Text Length

In [26]:
# Train weighted encodings
pred_df_full_throughput, pred_df_throughput, pred_df_full_worktime, pred_df_worktime, pred_df_full_agreement, pred_df_agreement, pred_df_full_textlength, pred_df_textlength, pred_df_full_special, pred_df_special = metadata_options.construct_weighted_dataframe(indices_train, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_df, pred_df_full)

In [27]:
# Test weighted encodings
pred_df_full_throughput_test, pred_df_throughput_test, pred_df_full_worktime_test, pred_df_worktime_test, pred_df_full_agreement_test, pred_df_agreement_test, pred_df_full_textlength_test, pred_df_textlength_test, pred_df_full_special_test, pred_df_special_test = metadata_options.construct_weighted_dataframe(indices_test, df_throughput, df_worktime, df_agreement, df_textlength, df_special, pred_test_df, pred_test_df_full)

## PC Agreement only

### Joint Model with one hot encoding 

In [28]:
print('Joint full model with one hot encoding, predicting deception, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_full_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_agreement, 
                               y=y_train_deception, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_full_agreement_test,y_test_deception), 
#                                callbacks=[models_nn.callback],
                               class_weight=deception_class_weight_dict)

Joint full model with one hot encoding, predicting deception, weighted by PC Agreement
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 5ms/step - loss: 0.6933 - acc: 0.8329 - f1_m: 0.8744 - precision_m: 0.9445 - recall_m: 0.8657 - val_loss: 0.690

143/143 [==============================] - 1s 4ms/step - loss: 0.6924 - acc: 0.5323 - f1_m: 0.6814 - precision_m: 0.9545 - recall_m: 0.5346 - val_loss: 0.6852 - val_acc: 0.5172 - val_f1_m: 0.6690 - val_precision_m: 0.9496 - val_recall_m: 0.5208
Epoch 30/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6921 - acc: 0.5009 - f1_m: 0.6531 - precision_m: 0.9520 - recall_m: 0.5028 - val_loss: 0.6858 - val_acc: 0.5084 - val_f1_m: 0.6607 - val_precision_m: 0.9515 - val_recall_m: 0.5107
Epoch 31/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6921 - acc: 0.2849 - f1_m: 0.4006 - precision_m: 0.9592 - recall_m: 0.2591 - val_loss: 0.6951 - val_acc: 0.3931 - val_f1_m: 0.5378 - val_precision_m: 0.9587 - val_recall_m: 0.3785
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6922 - acc: 0.3715 - f1_m: 0.5109 - precision_m: 0.9494 - recall_m: 0.3538 - val_loss: 0.6917 - val_acc: 0.4424 - val_f1_m: 0.5937 - val_precision_m: 0.9525 - val_r

In [29]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.5082010120397836, 0.5181381830461053, 0.4135049863613932, None)

In [30]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [1232 1042]


In [31]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [32]:
print('Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement')
joint_full_model = models_nn.create_joint_model(pred_df_agreement)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_agreement, 
                               y=y_train_rapport, 
                               epochs=32, 
                               batch_size=64, 
                               validation_data=(pred_df_agreement_test,y_test_rapport), 
#                                callbacks=[models_nn.callback], 
                               class_weight=rapport_class_weight_dict)

Joint full model with one hot encoding, predicting rapport, weighted by PC Agreement
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 1s 6ms/step - loss: 0.6762 - acc: 0.7099 - f1_m: 0.8148 - precision_m: 0.8961 - recall_m: 0.7498 - val_loss: 0.6699 

143/143 [==============================] - 0s 3ms/step - loss: 0.6540 - acc: 0.6945 - f1_m: 0.8014 - precision_m: 0.8966 - recall_m: 0.7274 - val_loss: 0.7079 - val_acc: 0.5932 - val_f1_m: 0.7285 - val_precision_m: 0.8612 - val_recall_m: 0.6348
Epoch 30/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6541 - acc: 0.6865 - f1_m: 0.7931 - precision_m: 0.8981 - recall_m: 0.7147 - val_loss: 0.7091 - val_acc: 0.5910 - val_f1_m: 0.7268 - val_precision_m: 0.8607 - val_recall_m: 0.6323
Epoch 31/32
143/143 [==============================] - 1s 4ms/step - loss: 0.6540 - acc: 0.6837 - f1_m: 0.7936 - precision_m: 0.8988 - recall_m: 0.7135 - val_loss: 0.6889 - val_acc: 0.6196 - val_f1_m: 0.7516 - val_precision_m: 0.8617 - val_recall_m: 0.6701
Epoch 32/32
143/143 [==============================] - 0s 3ms/step - loss: 0.6540 - acc: 0.6951 - f1_m: 0.8029 - precision_m: 0.8986 - recall_m: 0.7283 - val_loss: 0.6988 - val_acc: 0.6095 - val_f1_m: 0.7430 - val_precision_m: 0.8622 - val_r

In [33]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
# joint_predict_round = []
# for a in joint_predict:
#     joint_predict_round.append(np.argmax(a))
joint_predict_round = joint_predict.round()
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

(0.4916511307449137, 0.48580370171923376, 0.47661087999970186, None)

In [34]:
uni, cnt = np.unique(joint_predict_round, return_counts=True)
print(uni, cnt)

[0. 1.] [ 584 1690]
